In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as plt
import json
import time
import warnings

from bs4 import BeautifulSoup
from tqdm import tqdm
import requests

from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
import plotly.express as px
from tqdm.autonotebook import tqdm

import openpyxl 
import pycountry

import nltk
#nltk.download()


warnings.filterwarnings('ignore')


In [3]:
titles = pd.read_csv('../data/clean/titles.csv', encoding='utf-8', encoding_errors='ignore')
comments = pd.read_csv('../data/clean/com_group.csv', encoding='utf-8', encoding_errors='ignore')

## Test

In [4]:
# porcentaje de nulos de la tabla Titles
nan_cols = titles.isna().mean() * 100 
nan_cols[nan_cols>0]

Series([], dtype: float64)

In [5]:
# porcentaje de nulos de la tabla Titles
nan_cols = comments.isna().mean() * 100 
nan_cols[nan_cols>0]

Series([], dtype: float64)

In [6]:
pregunta1= 'Contemporáneas'
pregunta2= 'Acción'
pregunta3= 'Misterio'
pregunta4= ''
pregunta5= 'Hechos reales'
pregunta6= ('0','300')
pregunta7= 'Mucho'
pregunta8= 'hero'
pregunta9= 'Netflix'



In [7]:
respuestas = {
    'pregunta1': pregunta1,
    'pregunta2': pregunta2,
    'pregunta3': pregunta3,
    'pregunta4': pregunta4,
    'pregunta5': pregunta5,
    #'pregunta6': (duracion_minima, duracion_maxima),
    'pregunta7': pregunta7,
    'pregunta8': pregunta8,
    'pregunta9': pregunta9,
}

In [8]:
# 1. Filtro por antiguedad
if respuestas['pregunta1'] == 'Clásicas':
    df_filtrado = titles[titles['release_year'] <= 1990]
elif respuestas['pregunta1'] == 'Contemporáneas':
    df_filtrado = titles[titles['release_year'] > 1990]
elif respuestas['pregunta1'] == '¯\_(ツ)_/¯':
    df_filtrado = titles

In [9]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
84,ts20981,Survivor,SHOW,A reality show contest where sixteen or more c...,2000,TV-PG,48,reality,United States,42.0,tt0239195,7.4,24687,5419,7.4,Netflix,Jeff Probst
85,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
86,ts22176,Stargate SG-1,SHOW,The story of Stargate SG-1 begins about a year...,1997,TV-PG,44,"scifi, drama, action","Canada, United States",10.0,tt0118480,8.4,90196,88851,8.3,"Netflix, Amazon","Ben Browder, Amanda Tapping, Christopher Judge..."
87,tm15897,Se7en,MOVIE,Two homicide detectives are on a desperate hun...,1995,R,127,"crime, thriller, drama",United States,-1.0,tt0114369,8.6,1606270,51633,8.4,"Netflix, HBO","Brad Pitt, Morgan Freeman, Gwyneth Paltrow, Ke..."
88,ts21223,Pokémon,SHOW,"Join Ash accompanied by his partner Pikachu, a...",1997,TV-Y7,22,"action, scifi, animation, comedy, fantasy, family",Japan,25.0,tt14053732,7.4,198,115356,7.6,"Netflix, Amazon","Rica Matsumoto, Ikue Otani, Daiki Yamashita, K..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15311,tm948025,Hard Luck Love Song,MOVIE,"Jesse, a charismatic but down on his luck trou...",2023,R,104,"drama, thriller, romance, music",United States,-1.0,tt8709338,5.3,444,4299,5.3,Amazon,"Michael Dorman, Sophia Bush, RZA, Dermot Mulro..."
15312,tm811070,The Man on the Other Side,MOVIE,"It is 1974, and in the Cold War paranoia of Ea...",2023,R,85,"action, drama, thriller","United Kingdom, Singapore, Germany",-1.0,tt8000096,4.6,189,3415,2.0,Amazon,"Mari Bensel, James Carney, Nils Schulz, Jörg M..."
15313,ts382337,Marc Marquez: ALL IN,SHOW,Marc Márquez is facing the toughest decision o...,2023,unknown,33,documentation,Spain,1.0,tt26762647,8.1,74,2893,6.0,Amazon,"Marc Márquez, Dani Pedrosa, Jorge Lorenzo, Ale..."
15314,tm1292884,Hunt,MOVIE,"Arjun is the ACP of Hyderabad, who gets involv...",2023,R,129,"action, thriller, crime",India,-1.0,tt21946058,5.8,1269,4403,4.3,Amazon,"Sudheer Babu, Srikanth Meka, Bharath Srinivasa..."


In [10]:
# 2. Filtro por género favorito
if respuestas['pregunta2'] == 'Acción':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('action', case=False)]
elif respuestas['pregunta2'] == 'Drama':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('drama', case=False)]
elif respuestas['pregunta2'] == 'Comedia':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('comedy', case=False)]
elif respuestas['pregunta2'] == 'Documental':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('documentation', case=False)]
elif respuestas['pregunta2'] == 'Horror':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('horror', case=False) | df_filtrado['genres'].str.contains('thriller', case=False)]
elif respuestas['pregunta2'] == 'Romance':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('romance', case=False)]
elif respuestas['pregunta2'] == '¯\_(ツ)_/¯':
    pass

In [11]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
85,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
86,ts22176,Stargate SG-1,SHOW,The story of Stargate SG-1 begins about a year...,1997,TV-PG,44,"scifi, drama, action","Canada, United States",10.0,tt0118480,8.4,90196,88851,8.3,"Netflix, Amazon","Ben Browder, Amanda Tapping, Christopher Judge..."
88,ts21223,Pokémon,SHOW,"Join Ash accompanied by his partner Pikachu, a...",1997,TV-Y7,22,"action, scifi, animation, comedy, fantasy, family",Japan,25.0,tt14053732,7.4,198,115356,7.6,"Netflix, Amazon","Rica Matsumoto, Ikue Otani, Daiki Yamashita, K..."
93,ts20339,One Piece,SHOW,"Years ago, the fearsome Pirate King, Gol D. Ro...",1999,TV-14,23,"animation, action, comedy, drama, fantasy",Japan,21.0,tt0388629,8.8,117129,143457,8.8,Netflix,"Mayumi Tanaka, Kazuya Nakai, Akemi Okamura, Ka..."
96,ts34435,Cowboy Bebop,SHOW,"In 2071, roughly fifty years after an accident...",1998,TV-14,25,"animation, action, scifi, western, comedy, dra...",Japan,1.0,tt0213338,8.9,115448,35402,8.5,Netflix,"Koichi Yamadera, Unsho Ishizuka, Megumi Hayash..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15300,tm1302422,Kranti,MOVIE,"Kranti Narayan, a business tycoon in Europe is...",2023,R,163,"thriller, action, drama",India,-1.0,tt15373868,6.4,9962,46,8.0,Amazon,"Darshan Thoogudeepa, Rachita Ram, V. Ravichand..."
15301,ts363288,El Colapso,SHOW,It tells from the point of view of a diverse g...,2023,TV-14,165,"drama, action",Mexico,1.0,tt21613384,7.6,10,5369,7.3,Amazon,"Enrique Arrizon, Ana Valeria Becerril, Tiaré S..."
15302,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."
15312,tm811070,The Man on the Other Side,MOVIE,"It is 1974, and in the Cold War paranoia of Ea...",2023,R,85,"action, drama, thriller","United Kingdom, Singapore, Germany",-1.0,tt8000096,4.6,189,3415,2.0,Amazon,"Mari Bensel, James Carney, Nils Schulz, Jörg M..."


In [12]:
# 3. Filtro por tipo de trama favorita
sinonimos_misterio = ['mystery', 'enigma', 'puzzle', 'riddle', 'conundrum', 'secret', 'intrigue', 'clue', 'suspense']
sinonimos_aventura = ['thrill', 'adventure', 'expedition', 'journey', 'quest', 'explor', 'trek', 'voyage', 'clue', 'safari', 'exploration']
sinonimos_fantasia = ['fantasy', 'imagination', 'imaginary', 'enchant', 'magic', 'tail', 'fairy', 'myth', 'wonder', 'dream', 'illusion', 'super', 'superhero']

def buscar_sinonimos(critica, sinonimos):
    for sinonimo in sinonimos:
        if sinonimo in critica:
            return True
    return False

if respuestas['pregunta3'] == 'Misterio':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_misterio))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == 'Aventura':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_aventura))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == 'Fantasía':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_fantasia))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == '¯\_(ツ)_/¯':
    pass

In [13]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
1,ts34435,Cowboy Bebop,SHOW,"In 2071, roughly fifty years after an accident...",1998,TV-14,25,"animation, action, scifi, western, comedy, dra...",Japan,1.0,tt0213338,8.9,115448,35402,8.5,Netflix,"Koichi Yamadera, Unsho Ishizuka, Megumi Hayash..."
2,tm192037,Starship Troopers,MOVIE,"Set in the future, the story follows a young s...",1997,R,129,"scifi, action, thriller",United States,-1.0,tt0120201,7.3,291452,295,7.0,Netflix,"Casper Van Dien, Dina Meyer, Denise Richards, ..."
3,tm27395,Mission: Impossible II,MOVIE,With computer genius Luther Stickell at his si...,2000,PG-13,123,"thriller, action",United States,-1.0,tt0120755,6.1,337987,29392,6.1,"Netflix, Amazon","Tom Cruise, Dougray Scott, Thandiwe Newton, Vi..."
4,ts20983,Neon Genesis Evangelion,SHOW,"At the turn of the century, the Angels returne...",1995,TV-14,24,"drama, scifi, animation, action, fantasy, thri...",Japan,1.0,tt0112159,8.5,66629,4927,8.6,Netflix,"Megumi Ogata, Kotono Mitsuishi, Megumi Hayashi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,tm970846,The Handler,MOVIE,"After throwing a job, an ex-Marine seeks refug...",2021,R,85,"action, drama, thriller, crime",United States,-1.0,tt12525600,4.3,369,798,6.6,Amazon,"Chris Levine, Rachel Alig, Michael Pashan, Cat..."
1575,tm1297718,Die Hart,MOVIE,"Follows a fictionalized version of Kevin Hart,...",2023,R,85,"action, comedy, thriller",United States,-1.0,tt26198528,4.7,2380,3187531,6.3,Amazon,"Kevin Hart, John Travolta, Nathalie Emmanuel, ..."
1576,tm1190847,Varisu,MOVIE,"Vijay, The prodigal son of business tycoon Raj...",2023,unknown,170,"comedy, drama, action, romance",India,-1.0,tt11998558,6.2,31714,23632,6.6,Amazon,"Vijay, R. Sarathkumar, Prakash Raj, Shaam, Sri..."
1577,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."


In [14]:
# 4. Filtro por actor o actriz favorito
if respuestas['pregunta4'] != '':
    actor_favorito = respuestas['pregunta4']
    df_filtrado = df_filtrado[df_filtrado['actors'].str.contains(actor_favorito, case=False)]
else:
    pass

In [15]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
1,ts34435,Cowboy Bebop,SHOW,"In 2071, roughly fifty years after an accident...",1998,TV-14,25,"animation, action, scifi, western, comedy, dra...",Japan,1.0,tt0213338,8.9,115448,35402,8.5,Netflix,"Koichi Yamadera, Unsho Ishizuka, Megumi Hayash..."
2,tm192037,Starship Troopers,MOVIE,"Set in the future, the story follows a young s...",1997,R,129,"scifi, action, thriller",United States,-1.0,tt0120201,7.3,291452,295,7.0,Netflix,"Casper Van Dien, Dina Meyer, Denise Richards, ..."
3,tm27395,Mission: Impossible II,MOVIE,With computer genius Luther Stickell at his si...,2000,PG-13,123,"thriller, action",United States,-1.0,tt0120755,6.1,337987,29392,6.1,"Netflix, Amazon","Tom Cruise, Dougray Scott, Thandiwe Newton, Vi..."
4,ts20983,Neon Genesis Evangelion,SHOW,"At the turn of the century, the Angels returne...",1995,TV-14,24,"drama, scifi, animation, action, fantasy, thri...",Japan,1.0,tt0112159,8.5,66629,4927,8.6,Netflix,"Megumi Ogata, Kotono Mitsuishi, Megumi Hayashi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,tm970846,The Handler,MOVIE,"After throwing a job, an ex-Marine seeks refug...",2021,R,85,"action, drama, thriller, crime",United States,-1.0,tt12525600,4.3,369,798,6.6,Amazon,"Chris Levine, Rachel Alig, Michael Pashan, Cat..."
1575,tm1297718,Die Hart,MOVIE,"Follows a fictionalized version of Kevin Hart,...",2023,R,85,"action, comedy, thriller",United States,-1.0,tt26198528,4.7,2380,3187531,6.3,Amazon,"Kevin Hart, John Travolta, Nathalie Emmanuel, ..."
1576,tm1190847,Varisu,MOVIE,"Vijay, The prodigal son of business tycoon Raj...",2023,unknown,170,"comedy, drama, action, romance",India,-1.0,tt11998558,6.2,31714,23632,6.6,Amazon,"Vijay, R. Sarathkumar, Prakash Raj, Shaam, Sri..."
1577,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."


In [16]:

# 6. Filtro para determinar ficción
palabras_clave_ficcion = ['fiction', 'imaginary', 'fantasy', 'fictional', 'otherworld', 'extraterrestrial']
    
if respuestas['pregunta5'] == "Ficción":
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, palabras_clave_ficcion))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta5'] == "Hechos reales":
    comentarios_filtrados = comments[~comments['review'].apply(lambda x: buscar_sinonimos(x, palabras_clave_ficcion))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta5'] == '¯\_(ツ)_/¯':
    pass


In [17]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm27395,Mission: Impossible II,MOVIE,With computer genius Luther Stickell at his si...,2000,PG-13,123,"thriller, action",United States,-1.0,tt0120755,6.1,337987,29392,6.1,"Netflix, Amazon","Tom Cruise, Dougray Scott, Thandiwe Newton, Vi..."
1,tm116488,Natural Born Killers,MOVIE,Two victims of traumatized childhoods become l...,1994,R,118,"thriller, drama, crime, action",United States,-1.0,tt0110632,7.3,11,21933,7.1,Netflix,"Woody Harrelson, Juliette Lewis, Robert Downey..."
2,tm111163,Eraser,MOVIE,U.S. Marshall John Kruger erases the identitie...,1996,R,115,"thriller, action, drama, crime","United States, Canada",-1.0,tt0116213,6.1,110921,19437,6.0,Netflix,"Arnold Schwarzenegger, Vanessa Williams, James..."
3,tm187791,In the Line of Fire,MOVIE,Veteran Secret Service agent Frank Horrigan is...,1993,R,128,"drama, thriller, crime, action",United States,-1.0,tt0107206,7.2,102916,198,7.0,Netflix,"Clint Eastwood, John Malkovich, Rene Russo, Dy..."
4,tm65686,U.S. Marshals,MOVIE,U.S. Marshal Sam Gerard is accompanying a plan...,1998,PG-13,131,"thriller, crime, action",United States,-1.0,tt0120873,6.5,105286,24205,6.5,Netflix,"Tommy Lee Jones, Wesley Snipes, Robert Downey ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,tm970846,The Handler,MOVIE,"After throwing a job, an ex-Marine seeks refug...",2021,R,85,"action, drama, thriller, crime",United States,-1.0,tt12525600,4.3,369,798,6.6,Amazon,"Chris Levine, Rachel Alig, Michael Pashan, Cat..."
853,tm1297718,Die Hart,MOVIE,"Follows a fictionalized version of Kevin Hart,...",2023,R,85,"action, comedy, thriller",United States,-1.0,tt26198528,4.7,2380,3187531,6.3,Amazon,"Kevin Hart, John Travolta, Nathalie Emmanuel, ..."
854,tm1190847,Varisu,MOVIE,"Vijay, The prodigal son of business tycoon Raj...",2023,unknown,170,"comedy, drama, action, romance",India,-1.0,tt11998558,6.2,31714,23632,6.6,Amazon,"Vijay, R. Sarathkumar, Prakash Raj, Shaam, Sri..."
855,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."


In [18]:
# 6. Filtro por duración de la película
#df_filtrado = df_filtrado[(df_filtrado['duration'] >= duracion_minima) & (df_filtrado['duration'] <= duracion_maxima)]

In [19]:
# 7. Filtro por final feliz
sinonimos_final_feliz = ['happy ending', 'positive outcome', 'pleasant conclusion', 'satisfying resolution', 'joyful finale', 'contented ending', 'delightful outcome', 'cheerful conclusion', 'pleasant ending', 'uplifting finale']
    
if respuestas['pregunta7'] == 'Mucho':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_final_feliz))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta7'] == 'No':
    comentarios_filtrados = comments[~comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_final_feliz))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta7'] == '¯\_(ツ)_/¯':
    pass

In [20]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm116488,Natural Born Killers,MOVIE,Two victims of traumatized childhoods become l...,1994,R,118,"thriller, drama, crime, action",United States,-1.0,tt0110632,7.3,11,21933,7.1,Netflix,"Woody Harrelson, Juliette Lewis, Robert Downey..."
1,tm41508,First Sunday,MOVIE,Durell and LeeJohn are best friends and bumbli...,2008,PG-13,96,"comedy, action, crime, drama",United States,-1.0,tt0486578,4.5,10927,8809,5.6,Netflix,"Ice Cube, Katt Williams, Tracy Morgan, Loretta..."
2,tm80339,Naruto: Ninja Clash in the Land of Snow,MOVIE,Naruto is thrilled when he is sent on a missio...,2004,PG-13,90,"comedy, fantasy, animation, action",Japan,-1.0,tt0476680,6.6,6866,50604,7.2,Netflix,"Junko Takeuchi, Chie Nakamura, Noriaki Sugiyam..."
3,tm34924,Beast Stalker,MOVIE,Sergeant Tong is wracked with guilt after he u...,2008,R,110,"thriller, action",Hong Kong,-1.0,tt1330525,6.8,2219,4046,6.7,Netflix,"Nicholas Tse, Nick Cheung, Zhang Jingchu, Miao..."
4,tm120108,Kung Fu Panda 3,MOVIE,"Continuing his ""legendary adventures of awesom...",2016,PG,95,"action, animation, family, fantasy, drama, comedy","United States, China",-1.0,tt2267968,7.1,155678,91033,6.9,Netflix,"Jack Black, Bryan Cranston, Dustin Hoffman, An..."
5,ts39015,A Series of Unfortunate Events,SHOW,"The orphaned Baudelaire children face trials, ...",2017,TV-PG,47,"drama, action, comedy, family",United States,3.0,tt4834206,7.8,59959,55144,7.3,Netflix,"Neil Patrick Harris, Patrick Warburton, Malina..."
6,tm422491,Speed Kills,MOVIE,Speedboat racing champion and multimillionaire...,2018,R,102,"thriller, crime, drama, action","United States, Puerto Rico",-1.0,tt6749318,4.2,3244,14407,5.6,Netflix,"John Travolta, Katheryn Winnick, Jennifer Espo..."
7,ts84799,Victim Number 8,SHOW,An jihadist attack in the old town of Bilbao e...,2018,R,54,"crime, drama, action, thriller, fantasy",Spain,1.0,tt8500086,7.0,2030,6384,7.7,Netflix,"César Mateo, María de Nati, Verónika Moral, Iñ..."
8,tm356563,The Stolen,MOVIE,The story of a woman who must find her kidnapp...,2017,R,98,"western, drama, thriller, action","Germany, New Zealand, United Kingdom",-1.0,tt6441176,5.3,1330,615,5.5,Netflix,"Alice Eve, Jack Davenport, Graham McTavish, Ri..."
9,ts254656,Jurassic World: Camp Cretaceous,SHOW,Six teens attending an adventure camp on the o...,2020,TV-PG,24,"action, animation, scifi, family",United States,5.0,tt10436228,7.5,8022,259059,8.0,Netflix,"Jameela Jamil, Glen Powell, Ryan Potter, Raini..."


In [21]:
# 8. Filtro por tema, época o lugar favorito
def buscar_sinonimos(critica, sinonimos):
    for sinonimo in sinonimos:
        if sinonimo in critica:
            return True
    return False
    
if respuestas['pregunta8'] != '':
    tema = respuestas['pregunta8']
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, tema))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
else:
    pass

In [22]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm116488,Natural Born Killers,MOVIE,Two victims of traumatized childhoods become l...,1994,R,118,"thriller, drama, crime, action",United States,-1.0,tt0110632,7.3,11,21933,7.1,Netflix,"Woody Harrelson, Juliette Lewis, Robert Downey..."
1,tm41508,First Sunday,MOVIE,Durell and LeeJohn are best friends and bumbli...,2008,PG-13,96,"comedy, action, crime, drama",United States,-1.0,tt0486578,4.5,10927,8809,5.6,Netflix,"Ice Cube, Katt Williams, Tracy Morgan, Loretta..."
2,tm80339,Naruto: Ninja Clash in the Land of Snow,MOVIE,Naruto is thrilled when he is sent on a missio...,2004,PG-13,90,"comedy, fantasy, animation, action",Japan,-1.0,tt0476680,6.6,6866,50604,7.2,Netflix,"Junko Takeuchi, Chie Nakamura, Noriaki Sugiyam..."
3,tm34924,Beast Stalker,MOVIE,Sergeant Tong is wracked with guilt after he u...,2008,R,110,"thriller, action",Hong Kong,-1.0,tt1330525,6.8,2219,4046,6.7,Netflix,"Nicholas Tse, Nick Cheung, Zhang Jingchu, Miao..."
4,tm120108,Kung Fu Panda 3,MOVIE,"Continuing his ""legendary adventures of awesom...",2016,PG,95,"action, animation, family, fantasy, drama, comedy","United States, China",-1.0,tt2267968,7.1,155678,91033,6.9,Netflix,"Jack Black, Bryan Cranston, Dustin Hoffman, An..."
5,ts39015,A Series of Unfortunate Events,SHOW,"The orphaned Baudelaire children face trials, ...",2017,TV-PG,47,"drama, action, comedy, family",United States,3.0,tt4834206,7.8,59959,55144,7.3,Netflix,"Neil Patrick Harris, Patrick Warburton, Malina..."
6,tm422491,Speed Kills,MOVIE,Speedboat racing champion and multimillionaire...,2018,R,102,"thriller, crime, drama, action","United States, Puerto Rico",-1.0,tt6749318,4.2,3244,14407,5.6,Netflix,"John Travolta, Katheryn Winnick, Jennifer Espo..."
7,ts84799,Victim Number 8,SHOW,An jihadist attack in the old town of Bilbao e...,2018,R,54,"crime, drama, action, thriller, fantasy",Spain,1.0,tt8500086,7.0,2030,6384,7.7,Netflix,"César Mateo, María de Nati, Verónika Moral, Iñ..."
8,tm356563,The Stolen,MOVIE,The story of a woman who must find her kidnapp...,2017,R,98,"western, drama, thriller, action","Germany, New Zealand, United Kingdom",-1.0,tt6441176,5.3,1330,615,5.5,Netflix,"Alice Eve, Jack Davenport, Graham McTavish, Ri..."
9,ts254656,Jurassic World: Camp Cretaceous,SHOW,Six teens attending an adventure camp on the o...,2020,TV-PG,24,"action, animation, scifi, family",United States,5.0,tt10436228,7.5,8022,259059,8.0,Netflix,"Jameela Jamil, Glen Powell, Ryan Potter, Raini..."


In [23]:
# 9. Filtro por plataforma
if respuestas['pregunta9'] != '':
    plat_favorito = respuestas['pregunta9']
    df_filtrado = df_filtrado[df_filtrado['platform'].str.contains(plat_favorito, case=False)]
else:
    pass


In [24]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm116488,Natural Born Killers,MOVIE,Two victims of traumatized childhoods become l...,1994,R,118,"thriller, drama, crime, action",United States,-1.0,tt0110632,7.3,11,21933,7.1,Netflix,"Woody Harrelson, Juliette Lewis, Robert Downey..."
1,tm41508,First Sunday,MOVIE,Durell and LeeJohn are best friends and bumbli...,2008,PG-13,96,"comedy, action, crime, drama",United States,-1.0,tt0486578,4.5,10927,8809,5.6,Netflix,"Ice Cube, Katt Williams, Tracy Morgan, Loretta..."
2,tm80339,Naruto: Ninja Clash in the Land of Snow,MOVIE,Naruto is thrilled when he is sent on a missio...,2004,PG-13,90,"comedy, fantasy, animation, action",Japan,-1.0,tt0476680,6.6,6866,50604,7.2,Netflix,"Junko Takeuchi, Chie Nakamura, Noriaki Sugiyam..."
3,tm34924,Beast Stalker,MOVIE,Sergeant Tong is wracked with guilt after he u...,2008,R,110,"thriller, action",Hong Kong,-1.0,tt1330525,6.8,2219,4046,6.7,Netflix,"Nicholas Tse, Nick Cheung, Zhang Jingchu, Miao..."
4,tm120108,Kung Fu Panda 3,MOVIE,"Continuing his ""legendary adventures of awesom...",2016,PG,95,"action, animation, family, fantasy, drama, comedy","United States, China",-1.0,tt2267968,7.1,155678,91033,6.9,Netflix,"Jack Black, Bryan Cranston, Dustin Hoffman, An..."
5,ts39015,A Series of Unfortunate Events,SHOW,"The orphaned Baudelaire children face trials, ...",2017,TV-PG,47,"drama, action, comedy, family",United States,3.0,tt4834206,7.8,59959,55144,7.3,Netflix,"Neil Patrick Harris, Patrick Warburton, Malina..."
6,tm422491,Speed Kills,MOVIE,Speedboat racing champion and multimillionaire...,2018,R,102,"thriller, crime, drama, action","United States, Puerto Rico",-1.0,tt6749318,4.2,3244,14407,5.6,Netflix,"John Travolta, Katheryn Winnick, Jennifer Espo..."
7,ts84799,Victim Number 8,SHOW,An jihadist attack in the old town of Bilbao e...,2018,R,54,"crime, drama, action, thriller, fantasy",Spain,1.0,tt8500086,7.0,2030,6384,7.7,Netflix,"César Mateo, María de Nati, Verónika Moral, Iñ..."
8,tm356563,The Stolen,MOVIE,The story of a woman who must find her kidnapp...,2017,R,98,"western, drama, thriller, action","Germany, New Zealand, United Kingdom",-1.0,tt6441176,5.3,1330,615,5.5,Netflix,"Alice Eve, Jack Davenport, Graham McTavish, Ri..."
9,ts254656,Jurassic World: Camp Cretaceous,SHOW,Six teens attending an adventure camp on the o...,2020,TV-PG,24,"action, animation, scifi, family",United States,5.0,tt10436228,7.5,8022,259059,8.0,Netflix,"Jameela Jamil, Glen Powell, Ryan Potter, Raini..."
